In [1]:
import pandas as pd
import numpy as np
import random
from collections import Counter
import matplotlib.pyplot as plt
import time
import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.grid_search import RandomizedSearchCV
import sklearn.preprocessing
import sklearn.cross_validation
import sklearn.externals

import xgboost as xgb

%matplotlib inline

In [2]:
train = pd.read_csv('./train.csv').set_index("ID")
test = pd.read_csv('./test.csv').set_index("ID")

/home/ubuntu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (8,9,10,11,12,43,157,167,177,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
drop_cols = [c for c in train.columns if len(np.unique(train[c])) == 1]#Should later drop columns from train and test
#print drop_cols
train.drop(drop_cols, axis = 1, inplace = True)
test.drop(drop_cols, axis = 1, inplace = True)

/home/ubuntu/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [4]:
object_cols = train.columns[train.dtypes == 'object']
dates_cols =['VAR_0073', 'VAR_0075', 'VAR_0156', 'VAR_0157', 'VAR_0158', 'VAR_0159', 'VAR_0166', 'VAR_0167', 'VAR_0168', 'VAR_0169', 'VAR_0176', 'VAR_0177', 'VAR_0178','VAR_0179', 'VAR_0204', 'VAR_0217']

In [5]:
redundant_0008 = ['VAR_0008', 'VAR_0009', 'VAR_0010','VAR_0011','VAR_0012', 'VAR_0043','VAR_0044','VAR_0196', 'VAR_0229', ] #Contains redundant information
train.drop(redundant_0008, axis = 1, inplace = True)
test.drop(redundant_0008, axis = 1, inplace = True)

In [6]:
train['NaN'] = 0
test['NaN'] = 0

In [7]:
for c in test.columns: #Engineered Feature #1
    train['NaN'] += pd.isnull(train[c])
    test['NaN'] += pd.isnull(test[c])
    

In [8]:
def to_date(date_str):
    if pd.notnull(date_str):
        return datetime.datetime.strptime(date_str, '%d%b%y:%H:%M:%S')
    else:
        return date_str

In [9]:
for c in dates_cols:
    train[c] = train[c].apply(to_date)
    test[c] = test[c].apply(to_date)

In [10]:
diff = list(set(test.columns[test.dtypes == 'datetime64[ns]']) - set(train.columns[train.dtypes == 'datetime64[ns]'])) ##test got converted into date time format while train is still object format

In [11]:
train.drop(diff, axis = 1, inplace = True)
test.drop(diff, axis = 1, inplace = True)

In [12]:
dates_cols_fixed = train.columns[train.dtypes == 'datetime64[ns]']

In [13]:
#dates_cols_fixed

In [14]:
train['VAR_0204-VAR_0217'] = (train['VAR_0204'] - train['VAR_0217']).apply(lambda x: x.astype('timedelta64[D]')/np.timedelta64(1, 'D'))
test['VAR_0204-VAR_0217'] = (test['VAR_0204'] - test['VAR_0217']).apply(lambda x: x.astype('timedelta64[D]')/np.timedelta64(1, 'D'))

In [15]:
train['VAR_0204-VAR_0075'] = (train['VAR_0204'] - train['VAR_0075']).apply(lambda x: x.astype('timedelta64[D]')/np.timedelta64(1, 'D'))
test['VAR_0204-VAR_0075'] = (test['VAR_0204'] - test['VAR_0075']).apply(lambda x: x.astype('timedelta64[D]')/np.timedelta64(1, 'D'))

In [16]:
train['VAR_0073_ISNULL'] = pd.isnull(train['VAR_0073']).astype('int')
test['VAR_0073_ISNULL'] = pd.isnull(test['VAR_0073']).astype('int')

In [17]:
errors_dates = [] #Does not matter because will convert to float
for c in dates_cols_fixed: 
    try:
        train[c+'_YEAR'] = train[c].apply(lambda x: x.year)
        test[c+'_YEAR'] = test[c].apply(lambda x: x.year)
        train[c+'_MONTH'] = train[c].apply(lambda x: x.month)
        test[c+'_MONTH'] = test[c].apply(lambda x: x.month)
    except:
        errors_dates.append(c)

In [18]:
train.drop(dates_cols_fixed, axis = 1, inplace=True)
test.drop(dates_cols_fixed, axis = 1, inplace=True)

In [19]:
object_cols = train.columns[train.dtypes == 'object']


In [32]:
#random.seed(1)
#m,n = train.shape
#r = np.random.choice(m, size = 20000, replace=False)
#for c in object_cols:
#    print c, Counter(train.loc[r,c]).most_common(20)
#    print '\n\n'

#Unfixed dates ['VAR_0157','VAR_0167','VAR_0177']    

In [27]:
unfixed_dates = ['VAR_0157','VAR_0167','VAR_0177']   #previously dropped now added as null or not null

In [28]:
train.loc[:,unfixed_dates] = pd.isnull(train[unfixed_dates]).astype('int') #Added 10-14
test.loc[:,unfixed_dates] = pd.isnull(test[unfixed_dates]).astype('int') #Added 10-14

In [31]:
train[object_cols] #'VAR_0158' Also unknown date time Should addd to unfixed dates next iteration

,VAR_0001,VAR_0005,VAR_0158,VAR_0200,VAR_0202,VAR_0214,VAR_0216,VAR_0222,VAR_0226,VAR_0230,...,VAR_0325,VAR_0342,VAR_0352,VAR_0353,VAR_0354,VAR_0404,VAR_0466,VAR_0467,VAR_0493,VAR_1934
ID,,,,,,,,,,,,,,,,,,,,,
2,H,C,NaN,FT LAUDERDALE,BatchInquiry,NaN,DS,C6,False,False,...,-1,CF,O,U,O,CHIEF EXECUTIVE OFFICER,-1,-1,COMMUNITY ASSOCIATION MANAGER,IAPS
4,H,B,NaN,SANTEE,BatchInquiry,NaN,DS,C6,False,False,...,H,EC,O,R,R,-1,I,Discharged,-1,IAPS
5,H,C,NaN,REEDSVILLE,BatchInquiry,NaN,DS,C6,False,False,...,R,UU,R,R,-1,-1,-1,-1,-1,IAPS
7,H,C,NaN,LIBERTY,BatchInquiry,NaN,DS,C6,False,False,...,H,-1,R,R,-1,-1,-1,-1,-1,RCC
8,R,N,NaN,FRANKFORT,BatchInquiry,NaN,DS,C6,False,False,...,S,-1,R,U,O,-1,-1,-1,-1,BRANCH
14,R,C,NaN,SPRING,BatchInquiry,NaN,DS,C6,False,False,...,H,FE,U,R,R,-1,-1,-1,-1,IAPS
16,H,C,NaN,GRESHAM,BatchInquiry,NaN,DS,C6,False,False,...,S,DC,O,O,O,-1,-1,-1,-1,IAPS
20,R,B,NaN,WARNER ROBINS,BatchInquiry,NaN,DS,C6,False,False,...,S,FF,U,O,R,-1,-1,-1,-1,IAPS
21,R,N,NaN,SAN ANTONIO,BatchInquiry,NaN,DS,C6,False,False,...,S,-1,R,U,U,-1,-1,-1,-1,MOBILE


In [34]:
train.loc[290463, object_cols]

VAR_0001               H
VAR_0005               B
VAR_0158             NaN
VAR_0200    SHERMAN OAKS
VAR_0202    BatchInquiry
VAR_0214             NaN
VAR_0216              DS
VAR_0222              C6
VAR_0226           False
VAR_0230           False
VAR_0232            True
VAR_0236            True
VAR_0237              CA
VAR_0239           False
VAR_0274              CA
VAR_0283               S
VAR_0305              -1
VAR_0325              -1
VAR_0342              -1
VAR_0352              -1
VAR_0353              -1
VAR_0354              -1
VAR_0404              -1
VAR_0466              -1
VAR_0467              -1
VAR_0493              -1
VAR_1934            IAPS
Name: 290463, dtype: object

In [33]:
#train.drop(unfixed_dates, axis=1, inplace = True)
#test.drop(unfixed_dates, axis=1, inplace = True)

In [30]:
object_cols = train.columns[train.dtypes == 'object']


In [35]:
#train[object_cols].fillna(-1, inplace = True) # Not sure if executed cell
#test[object_cols].fillna(-1, inplace = True)

In [36]:
train.loc[:,object_cols] = train[object_cols].fillna(-10) #changed -1 to -10 as some columns have -1 as ??
test.loc[:,object_cols] = test[object_cols].fillna(-10)


In [37]:
object_cols = train.columns[train.dtypes == 'object']


In [38]:
encoders = {}
errors = []
for col in object_cols:
    if col not in dates_cols:
        encoders[col] = LabelEncoder()
        encoders[col].fit(pd.concat((train[col], test[col])))
        try:
            train[col] = encoders[col].transform(train[col])
            test[col] = encoders[col].transform(test[col])
        except:
            errors.append(col)
            #del train[col]
            #del test[col]


In [39]:
object_cols = train.columns[train.dtypes == 'object'] #Only 158 left
#print object_cols

In [41]:
train.drop(object_cols, axis=1, inplace = True) #This should not be needed in next iteration
test.drop(object_cols, axis=1, inplace = True)

In [42]:
#print train.info()
#print test.info()

In [43]:
X = train.drop('target', axis = 1).values
y = train.target.values
X_test = test.values

In [44]:
imputer = sklearn.preprocessing.Imputer(strategy='median') #Imputer should impute numerical variables
X = imputer.fit_transform(X)
X_test = imputer.transform(X_test)

In [45]:
scaler = sklearn.preprocessing.StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [47]:
xgb_clf = xgb.XGBClassifier(max_depth=20, learning_rate=.02, n_estimators=2250, subsample=.9, \
                            colsample_bytree=.8)

In [ ]:
cvs = sklearn.cross_validation.cross_val_score(xgb_clf, X, y, scoring = 'roc_auc', cv = 4)

In [ ]:
cvs

array([ 0.79179922,  0.79715366,  0.79188308,  0.79388739])

In [ ]:
xgb_clf.fit(X, y)
y_pred = xgb_clf.predict_proba(X_test)

In [ ]:
y_pred[:,1]

In [ ]:
# MAKING SUBMISSION
#xgb_clf2.save_model('0001.model')
#xgb_clf2.dump_model('dump.raw.txt', 'featmap.txt')

submission = pd.DataFrame(y_pred[:,1], index=test.index, columns=['target'])
submission.index.name = 'ID'
submission.to_csv('B_XGB-2_10_14_notebook.csv')

In [107]:
#sklearn.externals.joblib.dump(xgb_clf2, './models/XGB2.pkl')
#sklearn.externals.joblib.dump(encoders, './models/XGB2_encoders.pkl')

['./models/XGB2_encoders.pkl',
 './models/XGB2_encoders.pkl_01.npy',
 './models/XGB2_encoders.pkl_02.npy',
 './models/XGB2_encoders.pkl_03.npy',
 './models/XGB2_encoders.pkl_04.npy',
 './models/XGB2_encoders.pkl_05.npy',
 './models/XGB2_encoders.pkl_06.npy',
 './models/XGB2_encoders.pkl_07.npy',
 './models/XGB2_encoders.pkl_08.npy',
 './models/XGB2_encoders.pkl_09.npy',
 './models/XGB2_encoders.pkl_10.npy',
 './models/XGB2_encoders.pkl_11.npy',
 './models/XGB2_encoders.pkl_12.npy',
 './models/XGB2_encoders.pkl_13.npy',
 './models/XGB2_encoders.pkl_14.npy',
 './models/XGB2_encoders.pkl_15.npy',
 './models/XGB2_encoders.pkl_16.npy',
 './models/XGB2_encoders.pkl_17.npy',
 './models/XGB2_encoders.pkl_18.npy']

#*******************

In [108]:
#xgb2_load = sklearn.externals.joblib.load('./models/XGB2.pkl')

In [127]:
#xgb.XGBClassifier?

In [119]:
bst = xgb.XGBClassifier()

In [120]:
#grid_params = {'max_depth':[6, 10, 12],'learning_rate':[.008, .012], \
#               'n_estimators':[1500, 2500], 'subsample':[.7, .9], 'colsample_bytree':[.6, .8]}

In [121]:
grid_params = {'max_depth':[6, 10, 14],'learning_rate':[.010, .015], \
               'n_estimators':[1500, 2500], 'subsample':[.7, .9], 'colsample_bytree':[.6, .8]}

In [122]:
rand_gridsearch = RandomizedSearchCV(bst, param_distributions = grid_params, \
                                     n_iter = 20, scoring = 'roc_auc', cv = 4, verbose = True, random_state = 42)

In [124]:
rand_gridsearch.fit(X,y) #Not sure if this works?
best_xgb = rand_gridsearch.best_estimator_ 

In [128]:
print rand_gridsearch.best_params_

{'n_estimators': 25, 'subsample': 0.7, 'learning_rate': 0.012, 'colsample_bytree': 0.6, 'max_depth': 12}


In [126]:
for s in rand_gridsearch.grid_scores_:
    print s

mean: 0.74438, std: 0.00100, params: {'n_estimators': 15, 'subsample': 0.7, 'learning_rate': 0.012, 'colsample_bytree': 0.6, 'max_depth': 6}
mean: 0.74021, std: 0.00094, params: {'n_estimators': 15, 'subsample': 0.9, 'learning_rate': 0.008, 'colsample_bytree': 0.8, 'max_depth': 6}
mean: 0.75399, std: 0.00013, params: {'n_estimators': 15, 'subsample': 0.9, 'learning_rate': 0.008, 'colsample_bytree': 0.6, 'max_depth': 12}
mean: 0.75936, std: 0.00088, params: {'n_estimators': 25, 'subsample': 0.7, 'learning_rate': 0.012, 'colsample_bytree': 0.6, 'max_depth': 12}
mean: 0.75661, std: 0.00063, params: {'n_estimators': 25, 'subsample': 0.9, 'learning_rate': 0.012, 'colsample_bytree': 0.8, 'max_depth': 10}
mean: 0.74169, std: 0.00067, params: {'n_estimators': 15, 'subsample': 0.9, 'learning_rate': 0.012, 'colsample_bytree': 0.8, 'max_depth': 6}
mean: 0.74676, std: 0.00096, params: {'n_estimators': 25, 'subsample': 0.7, 'learning_rate': 0.012, 'colsample_bytree': 0.8, 'max_depth': 6}
mean: 0.75

In [131]:
y_pred = rand_gridsearch.predict_proba(X_test)


In [ ]:
y_pred = rand_gridsearch.predict_proba(X_test)

#MAKING SUBMISSION
#xgb_clf2.save_model('0001_grid.model')
#xgb_clf2.dump_model('dump_grid.raw.txt', 'featmap_grid.txt')

submission = pd.DataFrame(y_pred, index=test.index, columns=['target'])
submission.index.name = 'ID'
submission.to_csv('B_XGB_GridSearch.csv')
